In [2]:
import loompy as lp
import pandas as pd
loom = lp.connect("ccs_aucell_10kb_500bp_min10pct_repress_mingenes5_activatingonly.loom", mode='r+', validate=False)
auc_mtx = pd.DataFrame(loom.ca.RegulonsAUC, index=loom.ca.CellID)

In [16]:
auc_mtx.to_csv("ccs_10kb_500bp_min10pct_repress_mingenes5_activatingonly_aucmtx.csv")

In [4]:
from math import sqrt

import numpy as np
from scipy.spatial.distance import jensenshannon


def regulon_specificity_scores(auc_mtx, cell_type_series):
    """
    Calculates the Regulon Specificty Scores (RSS). [doi: 10.1016/j.celrep.2018.10.045]

    :param auc_mtx: The dataframe with the AUC values for all cells and regulons (n_cells x n_regulons).
    :param cell_type_series: A pandas Series object with cell identifiers as index and cell type labels as values.
    :return: A pandas dataframe with the RSS values (cell type x regulon).
    """

    cell_types = list(cell_type_series.unique())
    n_types = len(cell_types)
    regulons = list(auc_mtx.columns)
    n_regulons = len(regulons)
    rss_values = np.empty(shape=(n_types, n_regulons), dtype=np.float32)

    def rss(aucs, labels):
        # jensenshannon function provides distance which is the sqrt of the JS divergence.
        return 1.0 - jensenshannon(aucs / aucs.sum(), labels / labels.sum())

    for cidx, regulon_name in enumerate(regulons):
        for ridx, cell_type in enumerate(cell_types):
            rss_values[ridx, cidx] = rss(
                auc_mtx[regulon_name], (cell_type_series == cell_type).astype(int)
            )

    return pd.DataFrame(data=rss_values, index=cell_types, columns=regulons)

In [5]:
meta = pd.read_csv("ccs_meta.csv", index_col=0)

In [15]:
rss = regulon_specificity_scores(auc_mtx, meta["sub_all_onehis"])

In [17]:
rss.to_csv("ccs_10kb_500bp_min10pct_repress_mingenes5_activatingonly_rss_sub_all_onehis.csv")